In [1]:
### 웹브라우저 오픈을 위한 라이브러리
# - 브라우저 제어

from selenium import webdriver

### 수집된 html 정보에 html 타입으로 의미 부여 및 태그 정보 조회(정적)
from bs4 import BeautifulSoup

### 데이터 처리
import pandas as pd

In [2]:
### 크롬 브라우저 Open 함수
def getDriver() :
    return webdriver.Chrome()

In [3]:
### 멜론 차트 URL
url = "https://www.melon.com/chart/index.htm"

In [4]:
### 크롬 브라우저 open
driver = getDriver()

In [5]:
### URL 요청 후 html 정보 받아오기
driver.get(url)

In [6]:
### html 정보 추출하기
html = driver.page_source
html

'<html lang="ko" class="narrow-screen"><head>\n\t\t\n\t\n\t\n\t\n\t\n\n\t\n\n\t\n\t\n\t\n\t\n\n\t<meta http-equiv="Content-Type" content="text/html; charset=utf-8">\n\t<meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1">\n\n\t\n\n\t\n\n\t<title>멜론차트&gt;TOP100&gt;멜론</title>\n\t<meta name="keywords" content="음악서비스, 멜론차트, 멜론TOP100, 최신음악, 인기가요, 뮤직비디오, 앨범, 플레이어, 스트리밍, 다운로드, 아티스트플러스, 아티스트채널">\n\t<meta name="description" content="No.1 뮤직플랫폼 멜론! 최신 트렌드부터 나를 아는 똑똑한 음악추천까지!">\n\t<meta name="naver-site-verification" content="ee85ff6db1fa8f2226bcb671ecb2bcdbcffb6f8b">\n\t<meta name="google-site-verification" content="q4tbTQhmxa4La3OdNLsNOCxrJ_WV6lUlBFrFW4-HqQc">\n\t<meta property="fb:app_id" content="357952407588971">\n\t<meta property="og:title" content="Melon">\n\t<meta property="og:image" content="https://cdnimg.melon.co.kr/resource/image/web/common/logo_melon142x99.png">\n\t<meta property="og:description" content="음악이 필요한 순간, 멜론">\n\t<meta property="og:url" content="http://www.melon.

In [7]:
### HTML 타입으로 의미 부여
soup = BeautifulSoup(html, "html.parser")
soup

<html class="narrow-screen" lang="ko"><head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
<title>멜론차트&gt;TOP100&gt;멜론</title>
<meta content="음악서비스, 멜론차트, 멜론TOP100, 최신음악, 인기가요, 뮤직비디오, 앨범, 플레이어, 스트리밍, 다운로드, 아티스트플러스, 아티스트채널" name="keywords"/>
<meta content="No.1 뮤직플랫폼 멜론! 최신 트렌드부터 나를 아는 똑똑한 음악추천까지!" name="description"/>
<meta content="ee85ff6db1fa8f2226bcb671ecb2bcdbcffb6f8b" name="naver-site-verification"/>
<meta content="q4tbTQhmxa4La3OdNLsNOCxrJ_WV6lUlBFrFW4-HqQc" name="google-site-verification"/>
<meta content="357952407588971" property="fb:app_id"/>
<meta content="Melon" property="og:title"/>
<meta content="https://cdnimg.melon.co.kr/resource/image/web/common/logo_melon142x99.png" property="og:image"/>
<meta content="음악이 필요한 순간, 멜론" property="og:description"/>
<meta content="http://www.melon.com/chart/index.htm" property="og:url"/>
<meta content="website" property="og:type"/>
<meta content="멜론" pr

In [8]:
"""
* 제목 경로 : #lst50 > td:nth-child(6) > div > div > div.ellipsis.rank01 > span > a
             #lst100 > td:nth-child(6) > div > div > div.ellipsis.rank01 > span > a
             
* 가수 경로 : #lst50 > td:nth-child(6) > div > div > div.ellipsis.rank02 > a
             #lst100 > td:nth-child(6) > div > div > div.ellipsis.rank02 > a
"""

'\n* 제목 경로 : #lst50 > td:nth-child(6) > div > div > div.ellipsis.rank01 > span > a\n             #lst100 > td:nth-child(6) > div > div > div.ellipsis.rank01 > span > a\n             \n* 가수 경로 : #lst50 > td:nth-child(6) > div > div > div.ellipsis.rank02 > a\n             #lst100 > td:nth-child(6) > div > div > div.ellipsis.rank02 > a\n'

In [9]:
### 제목 조회해 보기
title_tags = soup.select("td:nth-child(6) > div > div > div.ellipsis.rank01 > span > a")
len(title_tags)

100

In [10]:
### 가수 조회해 보기
singer_tags = soup.select("td:nth-child(6) > div > div > div.ellipsis.rank02 > a")
len(singer_tags)

104

- 제목과 가수의 갯수가 다른 이유
    - 하나의 제목에 두명 이상(듀엣 이상)의 가수 이름이 포함된 경우에 가수의 a태그가 2개 이상씩 존재
    - 위 패턴으로는 사용 못함
- 제목과 가수의 모든 정보를 행단위로 가지고 와서 처리
    - < table > 태그는 행과열을 관리하는 영역으로 < tr >태그는 행을 담당, < td >태그는 열을 담당
    - 판다스에서 데이터프레임은 table, 행은 tr, 열은 td와 같은 의미

In [11]:
### 멜론 순위차트의 각 제목별 가수의 정보가 <table>로 되어 있음
# - 따라서 각 행<tr> 마다 제목과 가수가 있음
# - <tr> 태그 모두를 조회하여 행단위로 데이터 처리하면 제목과 가수들 추출 가능
# - 단 하나의 제목에 가수가 2명 이상인 경우에는 가수들을 하나의 문자열로 합치는 처리가 필요

In [12]:
### tr 태그 정보 모두 조회하기
songs = soup.select("tr")
len(songs)

101

In [13]:
### tr 태그의 총 갯수가 100개가 아닌 101개인 이유
# 표 제목이 있는 부분도 tr 태그가 있기 때문에 포함되어 추출됨
songs = songs[1 : ]
len(songs)

100

#### 100개 곡정보 중에 1개만 샘플링

In [14]:
### 0번째 값 : 0번째 곡정보, 즉 tr 태그 1개
song = songs[0]
song

<tr class="lst50" data-song-no="38444825" id="lst50">
<td><div class="wrap t_right"><input class="input_check" name="input_check" title="REBEL HEART 곡 선택" type="checkbox" value="38444825"/></div></td>
<td><div class="wrap t_center"><span class="rank">1</span><span class="none">위</span></div></td>
<!-- 차트순위 추가 -->
<td><div class="wrap">
<span class="rank_wrap" title="순위 동일">
<span class="bullet_icons rank_static"><span class="none">순위 동일</span></span>
<span class="none">0</span>
</span>
</div></td>
<td><div class="wrap">
<a class="image_typeAll" href="javascript:melon.link.goAlbumDetail('11689783');" title="IVE EMPATHY">
<img alt="IVE EMPATHY - 페이지 이동" height="60" onerror="WEBPOCIMG.defaultAlbumImg(this);" src="https://cdnimg.melon.co.kr/cm2/album/images/116/89/783/11689783_20250203110601_500.jpg/melon/resize/120/quality/80/optimize" width="60"/>
<span class="bg_album_frame"></span>
</a>
</div></td>
<td><div class="wrap">
<a class="btn button_icons type03 song_info" href="javascript:mel

In [15]:
### 1개 곡정보에서 [제목] 추출
title = song.select("div > div > div.ellipsis.rank01 > span > a")
title_name = title[0].text

title_name

'REBEL HEART'

In [16]:
### 1개 곡정보에서 [가수] 추출
singer = song .select("div > div > div.ellipsis.rank02 > a")
singer_name = singer[0].text

singer_name

'IVE (아이브)'

In [17]:
### 가수가 2명 이상인 경우 확인해보기
# - 현재 수집 기준 가수가 2명 이상인 행의 번호는 53번(리스트 인덱스는 52)
two_singer = songs[6]
singer = two_singer.select("div > div > div.ellipsis.rank02 > a")
singer


### 가수 2명 이상인 경우 하나의 문자열로 합쳐서 singer_nm 변수에 넣기
# - "경서예지 / 전건호 / ... "

singer_nm = []

# 각 가수 이름 추출
for idx in range(len(two_singer.select("div > div > div.ellipsis.rank02 > a"))):
    singer_name = two_singer.select("div > div > div.ellipsis.rank02 > a")[idx].get_text()
    singer_nm.append(singer_name)

# 가수 이름들을 하나의 문자열로 합침
singer_nm = " / ".join(singer_nm)

# 결과 확인
print(singer_nm)

IVE (아이브)


In [18]:
### 리스트 내에 값들을 문자열로 합치기
# - "구분자".join([데이터...]) 함수 사용

"/".join(["안녕", "넌", "누구니?"])

'안녕/넌/누구니?'

In [19]:
### 위 포문 한줄코딩
singer_nm = " / ".join([tag.text for tag in singer])

singer_nm

'IVE (아이브)'

In [20]:
if len(singer) >= 2 :
    singer_nm = " / ".join([tag.text for tag in singer])
else :
    singer_nm = singer[0].text

#### 100개 곡정보에서 곡제목과 가수명 조회하기

In [21]:
### 출력 예시
# [ 제목 ] : ㅁㅁㅁㅁㅁㅁ, [ 가수 ] : ㅁㅁㅁ

for idx in range(0, len(songs), 1):
    # print(idx)
    ### 각 행(tr태그)의 태그 가지고 오기(각 곡의 전체 정보가 있는 태그 입니다.)
    song = songs[idx]

    ### 1개 곡정보에서 [제목] 추출하기
    # select() : 리스트 타입으로 여러 태그들을 가지고 옵니다.
    # select_one() : 태그 자체 한개를 가지고 옵니다.
    title = song.select("div > div > div.ellipsis.rank01 > span > a")
    title_nm = title[0].text

    ### 1개 곡정보에서 [가수] 추출하기
    singer = song.select("div > div > div.ellipsis.rank02 > a")
    # singer_nm = singer[0].text
    singer_nm = ""
    if len(singer) >= 2 :
        singer_nm = "/".join([tag.text for tag in singer])
    else :
        singer_nm = singer[0].text

    ### 출력하기
    print(f"[곡제목] : {title_nm} , [가수명] : {singer_nm} ")



[곡제목] : REBEL HEART , [가수명] : IVE (아이브) 
[곡제목] : HOME SWEET HOME (feat. 태양, 대성) , [가수명] : G-DRAGON 
[곡제목] : 모르시나요(PROD.로코베리) , [가수명] : 조째즈 
[곡제목] : 나는 반딧불 , [가수명] : 황가람 
[곡제목] : Drowning , [가수명] : WOODZ 
[곡제목] : Whiplash , [가수명] : aespa 
[곡제목] : ATTITUDE , [가수명] : IVE (아이브) 
[곡제목] : APT. , [가수명] : 로제 (ROSÉ)/Bruno Mars 
[곡제목] : 오늘만 I LOVE YOU , [가수명] : BOYNEXTDOOR 
[곡제목] : Dash , [가수명] : PLAVE 
[곡제목] : toxic till the end , [가수명] : 로제 (ROSÉ) 
[곡제목] : HAPPY , [가수명] : DAY6 (데이식스) 
[곡제목] : POWER , [가수명] : G-DRAGON 
[곡제목] : RIZZ , [가수명] : PLAVE 
[곡제목] : 내게 사랑이 뭐냐고 물어본다면 , [가수명] : 로이킴 
[곡제목] : Chroma Drift , [가수명] : PLAVE 
[곡제목] : Island , [가수명] : PLAVE 
[곡제목] : 12:32 (A to T) , [가수명] : PLAVE 
[곡제목] : 사랑은 늘 도망가 , [가수명] : 임영웅 
[곡제목] : DRIP , [가수명] : BABYMONSTER 
[곡제목] : 소나기 , [가수명] : 이클립스 (ECLIPSE) 
[곡제목] : 천상연 , [가수명] : 이창섭 
[곡제목] : Welcome to the Show , [가수명] : DAY6 (데이식스) 
[곡제목] : Supernova , [가수명] : aespa 
[곡제목] : 너와의 모든 지금 , [가수명] : 재쓰비 (JAESSBEE) 
[곡제목] : 한 페이지가 될 수 있게 , [가수명] : DAY6 (데이

In [22]:
### 윈도우 창 닫기

driver.quit()